BPA Portfolio

In [ ]:
import pandas as pd
import os

In [ ]:
#Read data (second Excel sheet)
path = r'#input file path#'

df = pd.ExcelFile(path)
df2 = pd.read_excel(df, 'Portfolio Timeline')

# Print number of the unique investment and the shape of the original file
print('Unique investment(Original): ',df2['InvestmentCode'].nunique())
print('Shape of the dataset(Original): ',df2.shape)

In [ ]:
#Step 1. For Investments created before 2013, drop all their rows

investment_list = df2[df2['DateStamp'] < "2014-01-01"].loc[:,'InvestmentCode']

df2 = df2[~df2['InvestmentCode'].isin(investment_list)]

# Output the dataset after Step 1
operation = '_after 2013.xlsx'
root_ext = os.path.splitext(path)
outfile = root_ext[0] + operation
df2.to_excel(outfile, index=False, encoding='utf-8')

# Print number of the unique investment and the shape of the dataset after Step 1
print('Unique investment(Step 1): ',df2['InvestmentCode'].nunique())
print('Shape of the dataset(Step 1): ',df2.shape)

In [ ]:
#Step 2. Exclude rejected or merged investments by removing these or all related rows in Investment_Core_InvestmentStage

# 1. Exclude rejected or merged investments in Investment_Core_InvestmentStage
df21 = df2[~df2['Investment_Core_InvestmentStage'].isin(['Rejected','Merged'])]

# Output the dataset after Step 2 Version 1
operation = '_not merged or rejected.xlsx'
root_ext = os.path.splitext(path)
outfile = root_ext[0] + operation
df21.to_excel(outfile, index=False, encoding='utf-8')

# Print number of the unique investment and the shape of the dataset after Step 2 Version 1
print('Unique investment(Step 2 Version 1): ',df21['InvestmentCode'].nunique())
print('Shape of the dataset(Step 2 Version 1): ',df21.shape)

#2. Exclude rejected or merged investments and related rows in Investment_Core_InvestmentStage
RM = df2[df2['Investment_Core_InvestmentStage'].isin(['Rejected','Merged'])]

investment_list2 = RM.loc[:,'InvestmentCode']

df22 = df2[~df2['InvestmentCode'].isin(investment_list2)]

# Output the dataset after Step 2 Version 2
operation = '_not merged or rejected or related.xlsx'
root_ext = os.path.splitext(path)
outfile = root_ext[0] + operation
df22.to_excel(outfile, index=False, encoding='utf-8')

# Print number of the unique investment and the shape of the dataset after Step 2 Version 2
print('Unique investment(Step 2 Version 2): ',df22['InvestmentCode'].nunique())
print('Shape of the dataset(Step 2 Version 2): ',df22.shape)

In [ ]:
#Step 3. Remove investments that have never reached 'Physical Completion' in MilestoneName
# Remove single investment rows that have a milestone date (ActualDate, ForecastDate, TargetDate) on or after Oct 2021
# Remove single investment rows that do not have any forecast AND actuals on or after Oct 2021 then remove those 
# with a scenario forecast date on or after Oct 2021

pd.set_option('mode.chained_assignment', None) #Supress the warning massage due to chained indexing

def CompletedmileStone (df):
    "Remove single investment rows that have a milestone date(ActualDate, ForecastDate, TargetDate) on or after Oct 2021"
    df['date'] = df['Milestone_Core_ActualDate']
    for i in df.index:    
        if pd.isnull(df['date'][i]):
            df['date'][i] = df['Milestone_Core_ForecastDate'][i]
        if pd.isnull(df['date'][i]):
            df['date'][i] = df['Milestone_Core_TargetDate'][i]
    return df[df['date'] < "2021-10-01"]

def CompletedScenarioLastMonth(df):
    "Remove single investment rows that have scenario forecast date on or after Oct 2021 then remove those \
    with a scenario forecast date on or after Oct 2021"
    df[~(df['Milestone_Core_ForecastDate'].isnull() & df['Milestone_Core_ActualDate'].isnull())]
    return df[(df['Scenario__LastMonth'] <= "2021-10-01")]  

# Based on 21
# 1. Remove investments that have never reached 'Physical Completion' in MilestoneName
df_WithMilestone1 = df21.groupby('InvestmentCode').filter(lambda x: x['MilestoneName'].notnull().any())
# Print number of the unique investment and the shape of the dataset after Step 3 Version 1 Part 1
print('Unique investment(Step 3 Version 1 Part 1): ',df_WithMilestone1['InvestmentCode'].nunique())
print('Shape of the dataset(Step 3 Version 1 Part 1): ',df_WithMilestone1.shape)

# 2. Remove single investment rows that have a milestone date(ActualDate, ForecastDate, TargetDate) on or after Oct 2021
RM1 = CompletedmileStone(df_WithMilestone1)
investment_list21 = RM1.loc[:,'InvestmentCode']
df_NoFutureMilestone11 = df_WithMilestone1[df_WithMilestone1['InvestmentCode'].isin(investment_list21)]
df_NoFutureMilestone1 = df_NoFutureMilestone11.drop(columns=['date'])
# Print number of the unique investment and the shape of the dataset after Step 3 Version 1 Part 2
print('Unique investment(Step 3 Version 1 Part 2): ',df_NoFutureMilestone1['InvestmentCode'].nunique())
print('Shape of the dataset(Step 3 Version 1 Part 2): ',df_NoFutureMilestone1.shape)

# 3. Remove single investment rows that do not have any forecast AND actuals on or after Oct 2021 then remove those 
# with a scenario forecast date on or after Oct 2021
RM12 = CompletedScenarioLastMonth(df_NoFutureMilestone1)
investment_list211 = RM12.loc[:,'InvestmentCode']
df_NoFutureMilestone11 = df_NoFutureMilestone1[df_NoFutureMilestone1['InvestmentCode'].isin(investment_list211)]
# Print number of the unique investment and the shape of the dataset after Step 3 Version 1 Part 3
print('Unique investment(Step 3 Version 1 Part 3): ',df_NoFutureMilestone11['InvestmentCode'].nunique())
print('Shape of the dataset(Step 3 Version 1 Part 3): ',df_NoFutureMilestone11.shape)

# Output the dataset after Step 3 Version 1
operation = '_no Physical Completition1.xlsx'
root_ext = os.path.splitext(path)
outfile = root_ext[0] + operation
df_NoFutureMilestone11.to_excel(outfile, index=False, encoding='utf-8')

# Based on 22
# 1. Remove investments that have never reached 'Physical Completion' in MilestoneName
df_WithMilestone2 = df22.groupby('InvestmentCode').filter(lambda x: x['MilestoneName'].notnull().any())
# Print number of the unique investment and the shape of the dataset after Step 3 Version 2 Part 1
print('Unique investment(Step 3 Version 2 Part 1): ',df_WithMilestone2['InvestmentCode'].nunique())
print('Shape of the dataset(Step 3 Version 2 Part 1): ',df_WithMilestone2.shape)

# 2. Remove single investment rows that have a milestone date(ActualDate, ForecastDate, TargetDate) on or after Oct 2021
RM2 = CompletedmileStone(df_WithMilestone2)
investment_list22 = RM2.loc[:,'InvestmentCode']
df_NoFutureMilestone22 = df_WithMilestone2[df_WithMilestone2['InvestmentCode'].isin(investment_list22)]
df_NoFutureMilestone2 = df_NoFutureMilestone22.drop(columns=['date'])
# Print number of the unique investment and the shape of the dataset after Step 3 Version 2 Part 2
print('Unique investment(Step 3 Version 2 Part 2): ',df_NoFutureMilestone2['InvestmentCode'].nunique())
print('Shape of the dataset(Step 3 Version 2 Part 2): ',df_NoFutureMilestone2.shape)

# 3. Remove single investment rows that do not have any forecast AND actuals on or after Oct 2021 then remove those 
# with a scenario forecast date on or after Oct 2021
RM22 = CompletedScenarioLastMonth(df_NoFutureMilestone2)
investment_list212 = RM22.loc[:,'InvestmentCode']
df_NoFutureMilestone22 = df_NoFutureMilestone2[df_NoFutureMilestone2['InvestmentCode'].isin(investment_list212)]
# Print number of the unique investment and the shape of the dataset after Step 3 Version 2 Part 3
print('Unique investment(Step 3 Version 2 Part 3): ',df_NoFutureMilestone22['InvestmentCode'].nunique())
print('Shape of the dataset(Step 3 Version 2 Part 3): ',df_NoFutureMilestone22.shape)

# Output the dataset after Step 3 Version 2
operation = '_no Physical Completition2.xlsx'
root_ext = os.path.splitext(path)
outfile = root_ext[0] + operation
df_NoFutureMilestone22.to_excel(outfile, index=False, encoding='utf-8')

In [ ]:
# Step 4. Keep the only last submitted('ScenarioName') row for each investment's each stage('Investment_Core_InvestmentStage')

# 1. Based on 31
df41 = df_NoFutureMilestone11.groupby(['InvestmentCode','ScenarioName','Investment_Core_InvestmentStage'], as_index=False)\
.apply(lambda i:i.iloc[-1])
# Print number of the unique investment and the shape of the dataset after Step 4 Version 1
print('Unique investment(Step 4 Version 1): ',df41['InvestmentCode'].nunique())
print('Shape of the dataset(Step 4 Version 1): ',df41.shape)

# Output the dataset after Step 4 Version 1
operation = '_last submitted1.xlsx'
root_ext = os.path.splitext(path)
outfile = root_ext[0] + operation
df41.to_excel(outfile, index=False, encoding='utf-8')

# 2.Based on 32
df42 = df_NoFutureMilestone22.groupby(['InvestmentCode','ScenarioName','Investment_Core_InvestmentStage'], as_index=False)\
.apply(lambda i:i.iloc[-1])
# Print number of the unique investment and the shape of the dataset after Step 4 Version 2
print('Unique investment(Step 4 Version 2): ',df42['InvestmentCode'].nunique())
print('Shape of the dataset(Step 4 Version 2): ',df42.shape)

# Output the dataset after Step 4 Version 2
operation = '_last submitted2.xlsx'
root_ext = os.path.splitext(path)
outfile = root_ext[0] + operation
df42.to_excel(outfile, index=False, encoding='utf-8')